In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
from time import sleep

from ICARUS.database import DB

In [ ]:
# from ICARUS.Solvers.XFLR5.polars import read_polars_2d
from ICARUS.database import EXTERNAL_DB
# read_polars_2d(XFLRDB)

airfoils = DB.foils_db.get_airfoils()

print(f"Airfoils that are computed are:")
for i , key in enumerate(airfoils):
    print(f"{i}: {key}")
sleep(1)
# index = int(input("Select an airfoil: "))
index = 1
airfoil = list(airfoils)[index]

In [ ]:
from ICARUS.core.units import calc_reynolds
import numpy as np
from ICARUS.core.types import FloatArray

# PARAMETERS FOR ESTIMATION
chord_max: float = 0.4
chord_min: float = 0.4
u_max: float = 40
u_min: float = 5
viscosity: float = 1.56e-5

# MACH ESTIMATION
mach_max: float = 0.08
# mach_min: float = calc_mach(10, speed_of_sound)
# mach: FloatArray = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
reynolds: FloatArray = np.linspace(
    start=reynolds_min,
    stop=reynolds_max,
    num=5,
)

# ANGLE OF ATTACK SETUP
aoa_min: float = -10
aoa_max: float = 10
num_of_angles: int = int((aoa_max - aoa_min) * 2 + 1)
angles: FloatArray = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

# Transition to turbulent Boundary Layer
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}
ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
Ncrit = 9

In [19]:
from ICARUS.airfoils.airfoil import Airfoil

airfoil = Airfoil.naca('2412')
airfoil.plot()
# airfoil.xx

In [ ]:
from ICARUS.computation.solvers.Xfoil.analyses.angles import single_reynolds_run, single_reynolds_run_seq

a = single_reynolds_run(
    Reyn = reynolds[-1],
    MACH = MACH,
    AoAmin = 0,
    AoAmax = aoa_max,
    AoAstep= 0.5,
    airfoil = airfoil,
    solver_options= {
    "Ncrit" : Ncrit,
    "ftrip_low" : 0.001,
    "ftrip_up" : 0.001,
    "print": True
    }
)
# numpy remove nan
a = a[~np.isnan(a).any(axis=1)]

In [ ]:
import matplotlib.pyplot as plt
import scipy

# Make 4 subplots for the 4 different plots
fig, axs = plt.subplots(2, 2)

# Plot the Cl vs AoA
axs[0, 0].plot(a[:, 0], a[:, 1])
# # Make a linear interpolation to extend the curve of the Cl vs AoA to 5 degrees dotted
# # fit a curve to the data points using a 3rd order polynomial
# z = np.polyfit(a[:, 0], a[:, 1], 3)
# # construct the polynomial
# p = np.poly1d(z)
# # construct the x values for the linear interpolation
# x = np.linspace(a[:, -1][0], 5, 100)
# # construct the y values for the linear interpolation
# y = p(x)
# # plot the linear interpolation
# axs[0, 0].plot(x, y, linestyle="--", color="black")

axs[0, 0].set_title("Cl vs AoA")
axs[0, 0].set_xlabel("AoA [deg]")
axs[0, 0].set_ylabel("Cl [-]")
axs[0, 0].grid()

# Plot the Cd vs AoA
axs[0, 1].plot(a[:, 0], a[:, 2])
axs[0, 1].set_title("Cd vs AoA")
axs[0, 1].set_xlabel("AoA [deg]")
axs[0, 1].set_ylabel("Cd [-]")
axs[0, 1].grid()

# Plot the Cm vs AoA
axs[1, 0].plot(a[:, 0], a[:, 3])
axs[1, 0].set_title("Cm vs AoA")
axs[1, 0].set_xlabel("AoA [deg]")
axs[1, 0].set_ylabel("Cm [-]")
axs[1, 0].grid()

# Plot the Cl/Cd vs AoA
axs[1, 1].plot(a[:, 0], a[:, 1]/ a[:, 2])
axs[1, 1].set_title("Cl/Cd vs AoA")
axs[1, 1].set_xlabel("AoA [deg]")
axs[1, 1].set_ylabel("Cl/Cd [-]")
axs[1, 1].grid()

# Make the plots look nice
fig.tight_layout()

# Show the plots
plt.show()
